# **Objetivo del Cuaderno Jupyter**
- Procesar el archivo `credits.csv` y dejar la información mas consumible en los archivos `cast.parquet` y `crew.parquet`, por cualquier otro departamento de la organización.
- Descomponer en tablas mas específicas como `actores` y `directores` que sirvan de apoyo para satisfacer la necesidad de las `funciones 5 y 6 de la Api`, así como `aportar información al modelo de recomendación` .


In [6]:
import polars as pl

***
# **A Pie**

### **Funcion `get_fields_from_line`**

In [2]:
def get_fields_from_line(line:str) -> tuple[list, str]:
    """get_fields_from_line: 
        Funcion que acepta una linea del archivo 'credits.csv' y devuelve una Lista o Varias Listas y un valor string
            Las Listas contienen a su vez Muchos diccionarios:
            .- Estos diccionarios pueden ser contentivos de data para  `cast` o para `crew`
            .- NOTA: El Codigo que llama a esta función en la siguiente celda se encarga de 
                     separar los diccionarios
            El string:
            .- El `id` de la movie con que estan relacionados los diccionarios de las listas
                Nota: El proceso de casteo a data tipo Int lo hace automaticamente la libreria 
                        utilizada para convertir la data en un Dataframe Luego

    Puntos a tomar en cuenta:
        .- La Mayoría de las `line` que recibe la funcion tiene la siguiente estructura:
            "[{dict de cast}, {dict de cast}, etc..]""[{dict de crew},{dict de crew}, etc]",numero_id      
        .- En Ocaciones `line` sólo viene con una(1) lista y el numero_id
        .- En Ocaciones `line` ademas de las dos listas con data, trae listas vacias `[]`, al pricipio,
           al final o intercaladas entre las otras listas
        .- Aunque las listas estan encerradas entre comillas dobles `"` es muy probable conseguir comillas
           dobles dentro de los diccionarios
        .- Los diccionarios como cualquier diccionarios están compuestos de Clave(key) y Valor(Value), encerrados 
           entre comillas simples `'` pero con MUCHISIMA FRECUENCIA entre esas comillas simples `'`, hay  comillas 
           dobles `"`,  dos comillas dobles repetidas `""`, estas últimas a veces aparecen delimitando el contenido
           de un value de alguna clave de un diccionario (al principio y al final del value), pero hay veces aparecen
           dentro del contenido del value.  Tambien aparecen exporádicamente caracteres de escape acompañados 
           con `\\′ porsupuesto.

        Nota: Dada las irregularidades que contiene la data, preferí hacer la función customizada y no dejarle a las
              librerias de Python como `Pandas` o `Polars` la carga directa de los datos, ya que considero complejo
              y con tendencia a errores la Inferencia de algun esquema de datos, dado que el esquema de la data
              presenta muchas irregularidades, lo que se puede transformar en carga errónea de datos.   

           
    Lógica de la Función:
        La lógica de la función es recorrer uno a uno los caracteres de la `line` con una variable llamada `index`
        y a medida que avanza ir indicando con variables de nombre apropiado si el index se encuentra dentro de datos
        que se espera esten entre comillas dobles `"`, corchetes `[]`, comillas simples `'`, etc.

    Variables BOOLEANAS A tener en cuenta:
        .- open_doubles:  Indica sí el index se encuentra dentro de datos que se espera estén entre comillas 
                          dobles (") -> comillas dobles abiertas o cerradas (para su facil lectura en los condicionales)  
        .- open_singles:  Indica sí el index se encuentra dentro de datos que se espera estén entre comillas simples (')
        .- open_brackets: Indica sí el index se encuentra dentro de datos que se espera estén entre corchetes ([]) 
        .- open_two_doubles: Indica sí el index se encuentra dentro de datos que se espera estén entre dos comillas dobles ("")

    Args:
        line (str): Una Linea completa del archivo `credits.csv`

    Returns:
        tuple[list, str]: Una Lista de Diccionarios y el string que identifica el id de la movie
    """
    # Tengo que convertir a `line` en una lista para poder manipularla y en ocaciones corregirle 
    # pequeño detalles. 
    line_type_list:list = list(line)
    cadenas:list = []
    movie_id_str:str = ''
    #  Variable que utilizaré para marcar el inicio de la cadena de caracteres cuando hago el slice [:]
    first:int = 0  
    open_doubles:bool = False  
    open_singles:bool = False  
    open_brackets:bool = False 
    open_two_doubles:bool = False  
    for index in range(len(line_type_list)):
        if line_type_list[index] == '"' and not open_brackets and not open_doubles and not open_singles:
            open_doubles = True
            first = index
        elif line_type_list[index] == '[' and open_doubles:
            open_brackets = True
        elif line_type_list[index] == "'" and open_brackets and open_doubles and open_singles:
            if line_type_list[index+1] in [',',':', '}']:
                open_singles = False
            else:
                # Hay veces se encuentra comilla simple ' donde no deberían estar. El caso Nombres o expresiones en Ingles 
                # ejemplo: O'Neal ó Peter's house.  En este caso aunque esas comillas simples son parte de la información
                # rompen el codigo. Las sustituyo por backtick ` para evitar que rompan el codigo ahora y luego cuando tenga
                # la data procesada en un Dataframe los vuelvo a reemplazar por la comilla original.
                line_type_list[index] = '`'
        elif line_type_list[index] == "'" and open_brackets and open_doubles and not open_singles:     
            if line_type_list[index-1].isalpha() or open_two_doubles:
                line_type_list[index] = '`'
            else:
                open_singles = True
        elif line_type_list[index] =='"' and open_brackets and open_doubles and open_singles:
            line_type_list[index] = ' '
        elif line_type_list[index] == '"' and open_brackets and open_doubles and not open_singles:
            if line_type_list[index+1] == '"':
                line_type_list[index] = ' '
                open_two_doubles = False if open_two_doubles else True
            else:
                line_type_list[index] = "'"
        elif line_type_list[index] == ']' and not open_singles and open_doubles and not open_two_doubles:
            open_brackets = False
        elif line_type_list[index] == '"' and not open_singles and not open_brackets and open_doubles:
            open_doubles = False
            cadena = line_type_list[first:index+1]
            cadena = eval(''.join(cadena))
            cadenas.append(cadena) 
            first = index + 2
        elif line_type_list[index] == '"' and open_singles:
            line_type_list[index] = ' '
        elif line_type_list[index] == '\\':
            line_type_list[index] = ' '
            if line_type_list[index+1] == 't' or line_type_list[index+1] == 'T':
                line_type_list[index+1] = ' '            
        elif index == len(line_type_list) - 1:
            for i in range(len(line_type_list)):
                if line_type_list[index - i] == ',':
                    first = index - i + 1
                    break
            movie_id_str = line_type_list[first:]
            movie_id_str = ''.join(movie_id_str)
            first = 0

    return (cadenas, movie_id_str)

**Ya tengo una función que extrae de una linea, los diccionarios de `cast`, `crew` y el `id` del movie al cual están relacionados dichos diccionarios. Ahora debo abrir el archivo como texto y mandar cada linea a la función, separar los diccionarios de `cast` y `crew` y a la vez asociarlos al `id` del movie correspondiente** 

# **ALTO**
### **POR FAVOR LEA ESTE MENSAJE ANTES DE CONTINUAR**

Para poder continuar Necesitamos el archivo `credits.csv` que nos dieron al empezar el Proyecto:<br>

**OPCIONES**
- Sí dispone del archivo `credits.csv` coloquelo en la carpeta `datasets_inicial` de este proyecto `NO EJECUTE LA CELDA 6` Osea la siguiente celda.
- Sí por el contrario NO DISPONE del archivo, entonces ejecute la siguiente celda para descargarlo.

In [1]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=11zFVAMts_ZWF4mIA6FYtf-ITD2RzQPRs'

gdown.download(url, 'datasets_inicial/credits.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=11zFVAMts_ZWF4mIA6FYtf-ITD2RzQPRs
From (redirected): https://drive.google.com/uc?export=download&id=11zFVAMts_ZWF4mIA6FYtf-ITD2RzQPRs&confirm=t&uuid=7511f86e-c98e-43b1-a781-7d2ba952745e
To: d:\BOOTCAMP-HENRY\MODULO-07-LABS\datasets_inicial\credits.csv
100%|██████████| 190M/190M [00:35<00:00, 5.33MB/s] 


'datasets_inicial/credits.csv'

Recorro el archivo como si fuera texto y cada linea lo mando a la función

In [3]:
data_cast = []  #Variable donde alojaré todos los diccionarios de cast
data_crew = []  #Variable donde alojaré todos los diccionarios de crew

with open("datasets_inicial/credits.csv", 'r', encoding='utf-8') as file:
    next(file) # Esto se simplemente para Saltar el encabezado
    lines = 0
    for line in file:
        lines += 1
        line = line.strip()
        list_of_strings, movie_id_str = get_fields_from_line(line=line) #Llamo a la funcion y le paso cada linea
        # Es posible recibir una lista vacia.  Hay lineas del dataset que son solo `[],[]`
        if len(list_of_strings) > 0:
            # De recibir algo es posible recibir una o dos Listas de Diccionarios
            for i in range(len(list_of_strings)):
                # la funcion se encargo de hacer pequeños ajustes y ahora utilizo el eval() sin que se rompa
                list_of_dict = eval(list_of_strings[i])
                for dictionary in list_of_dict:
                    # Indistintamente si el dict es de `cast` o de `crew` hay que enlazarlo con `movie_id`
                    dictionary['movie_id'] = int(movie_id_str)               
                    if 'cast_id' in dictionary.keys():
                        data_cast.append(dictionary)
                    else:
                        data_crew.append(dictionary)
    file.close()

### **Probando como viene la Data de `get_fields_from_line`**

In [4]:
# Abajo Un ejemplo de un Diccionario de CAST
print(f'Longitud de La data de cast  {len(data_cast)}')
data_cast[0]

Longitud de La data de cast  562474


{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg',
 'movie_id': 862}

In [4]:
# Abajo Un ejemplo de un Diccionario de CREW
print(f'Longitud de La data de crew  {len(data_crew)}')
data_crew[10]

Longitud de La data de crew  464314


{'credit_id': '52fe4284c3a36847f8024f91',
 'department': 'Art',
 'gender': 2,
 'id': 7883,
 'job': 'Art Direction',
 'name': 'Ralph Eggleston',
 'profile_path': '/uUfcGKDsKO1aROMpXRs67Hn6RvR.jpg',
 'movie_id': 862}

Como se Observa recopile todos los diccionarios y los introduje en su respectiva Lista (`data_cast` o `data_crew`) y ahora es mas sencillo crear los Dataframes

***
#### **Trabajando la tabla de Actores, sacando la data de los diccionarios de `cast`**

In [7]:
cast = pl.DataFrame(data=data_cast)
cast.head()

cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
i64,str,str,i64,i64,str,i64,str,i64
14,"""Woody (voice)""","""52fe4284c3a36847f8024f95""",2,31,"""Tom Hanks""",0,"""/pQFoyx7rp09CJTAb932F2g8Nlho.j…",862
15,"""Buzz Lightyear (voice)""","""52fe4284c3a36847f8024f99""",2,12898,"""Tim Allen""",1,"""/uX2xVf6pMmPepxnvFWyBtjexzgY.j…",862
16,"""Mr. Potato Head (voice)""","""52fe4284c3a36847f8024f9d""",2,7167,"""Don Rickles""",2,"""/h5BcaDMPRVLHLDzbQavec4xfSdt.j…",862
17,"""Slinky Dog (voice)""","""52fe4284c3a36847f8024fa1""",2,12899,"""Jim Varney""",3,"""/eIo2jVVXYgjDtaHoF19Ll9vtW7h.j…",862
18,"""Rex (voice)""","""52fe4284c3a36847f8024fa5""",2,12900,"""Wallace Shawn""",4,"""/oGE6JqPP2xH4tNORKNqxbNPYi7u.j…",862


In [5]:
# Elimino Espacios vacios al principio y final de cada campo String
cast = cast.with_columns([
    pl.col('name').str.to_uppercase().str.strip_chars().alias('name'),
    pl.col('character').str.strip_chars().alias('character'),
    pl.col('profile_path').str.strip_chars().alias('profile_path'),   
])

# Sustituyo los backtick ` por comilla ' para dejar la informacion como venía originalmente, ya en este punto 
# la comilla no me rompe el codigo. Nota: La explicación de esto esta en un comentario de 4 lineas en la celda 2
cast = cast.with_columns([
    pl.col('character').str.replace("`", "'"),
    pl.col('profile_path').str.replace("`", "'"),
    pl.col('name').str.replace("`", "'") ,
])

Mostrando cuantos Nulos y Datos en Blanco me vinieron en la data

In [6]:
for column in cast.columns:
    mask = (cast[column] == "") if cast[column].dtype == 'String' else (cast[column] == 0)
    print(f'Cantidad de Nulos --> {cast[column].is_null().sum()}  Columna --> {column}')
    print(f'Cantidad de "" ó 0 --> {cast.filter(mask).shape[0]}') 
    print('*'*50)

Cantidad de Nulos --> 0  Columna --> cast_id
Cantidad de "" ó 0 --> 2846
**************************************************
Cantidad de Nulos --> 0  Columna --> character
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> credit_id
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> gender
Cantidad de "" ó 0 --> 223964
**************************************************
Cantidad de Nulos --> 0  Columna --> id
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> name
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> order
Cantidad de "" ó 0 --> 36747
**************************************************
Cantidad de Nulos --> 173856  Columna --> profile_path
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos

Grabaré a un archivo `cast.parquet` este dataframe, por sí en un futuro necesito alguna información adicional a los dataframes `actores_df` y `directores_df` que crearé mas adelante

In [7]:
cast.write_parquet('data/cast.parquet')

En esta data `cast` aparecen registros de Actores por cada pelicula en que a participado. Para responder las preguntas planteadas `para la Api (especificamente la 5)`, y llevarme `información relevante para el modelo`. Me llevaré
- Nombre del Actor: `actor`
- Lista de los id de las peliculas en que participo: `movies`
- Número de películas en las que participó: `number_of_movies`
- El id del actor: `id_actor` --> Este campo, por ahora, creo no es necesario, me lo llevo por si lo necesito mas adelante, o mas adelante en la organizacion necesitan hacer un datawarehouse con esta información.

In [8]:
# En data_actores metere uno a uno los distintos actores
data_actores = []
actores = cast['name'].unique().to_list() #Listado de Actores

for actor in actores:
    row_actores = {}
    mask = cast['name'] == actor # Filtro a `cast` por nombre de actor
    id_actor = cast.filter(mask)['id'].unique()[0] # Tomo el `id` del actor
    movies_ids = cast.filter(mask)['movie_id'].unique().to_list() # Tomo los `id` de las movies
    number_of_movies = len(movies_ids)
    row_actores['id_actor'] = id_actor
    row_actores['actor'] = actor
    row_actores['movies'] = movies_ids
    row_actores['number_of_movies'] = number_of_movies
    data_actores.append(row_actores)

In [9]:
actores_df = pl.DataFrame(data=data_actores)

actores_df.head()

id_actor,actor,movies,number_of_movies
i64,str,list[i64],i64
120646,"""NINO FRASSICA""","[25846, 37710, … 320316]",8
142964,"""CARMI MARTIN""",[40471],1
1072363,"""BEATRICE CURTIS""","[3083, 24807, … 149793]",4
1668887,"""DORIS MORTON""",[33810],1
1564286,"""KRZYSZTOF CHMIELEWSKI""",[375742],1


In [10]:
actores_df.write_parquet('data/actores.parquet')

***
#### **Trabajando la tabla Directores, sacando la información de los diccionarios de `crew`**

In [11]:
crew = pl.DataFrame(data=data_crew)

print(crew.shape)
crew.head()

(464314, 8)


credit_id,department,gender,id,job,name,profile_path,movie_id
str,str,i64,i64,str,str,str,i64
"""52fe4284c3a36847f8024f49""","""Directing""",2,7879,"""Director""","""John Lasseter""","""/7EdqiNbr4FRjIhKHyPPdFfEEEFG.j…",862
"""52fe4284c3a36847f8024f4f""","""Writing""",2,12891,"""Screenplay""","""Joss Whedon""","""/dTiVsuaTVTeGmvkhcyJvKp2A5kr.j…",862
"""52fe4284c3a36847f8024f55""","""Writing""",2,7,"""Screenplay""","""Andrew Stanton""","""/pvQWsu0qc8JFQhMVJkTHuexUAa1.j…",862
"""52fe4284c3a36847f8024f5b""","""Writing""",2,12892,"""Screenplay""","""Joel Cohen""","""/dAubAiZcvKFbboWlj7oXOkZnTSu.j…",862
"""52fe4284c3a36847f8024f61""","""Writing""",0,12893,"""Screenplay""","""Alec Sokolow""","""/v79vlRYi94BZUQnkkyznbGUZLjT.j…",862


In [12]:
# Elimino Espacios vacios al principio y final de cada campo String
crew = crew.with_columns([
    pl.col('credit_id').str.strip_chars().alias('credit_id'),
    pl.col('department').str.strip_chars().alias('department'),
    pl.col('job').str.to_uppercase().str.strip_chars().alias('job'),
    pl.col('name').str.to_uppercase().str.strip_chars().alias('name'),
    pl.col('profile_path').str.strip_chars().alias('profile_path'),
])

# Sustituyo los backtick ` por comilla ' para dejar la informacion como venía originalmente, ya en este punto 
# la comilla no me rompe el codigo. Nota: La explicación de esto esta en un comentario de 4 lineas en la celda 2
crew = crew.with_columns([
    pl.col('credit_id').str.replace("`", "'"),
    pl.col('department').str.replace("`", "'"),
    pl.col('job').str.replace("`", "'"),
    pl.col('name').str.replace("`", "'"),
    pl.col('profile_path').str.replace("`", "'"),
])

Revisando Nulos e información vacia ó 0

In [13]:
for column in crew.columns:
    mask = (crew[column] == "") if crew[column].dtype == 'String' else (crew[column] == 0)
    print(f'Cantidad de Nulos --> {crew[column].is_null().sum()}  Columna --> {column}')
    print(f'Cantidad de "" ó 0 --> {crew.filter(mask).shape[0]}') 
    print('*'*50)

Cantidad de Nulos --> 0  Columna --> credit_id
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> department
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> gender
Cantidad de "" ó 0 --> 272319
**************************************************
Cantidad de Nulos --> 0  Columna --> id
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> job
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> name
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 369216  Columna --> profile_path
Cantidad de "" ó 0 --> 0
**************************************************
Cantidad de Nulos --> 0  Columna --> movie_id
Cantidad de "" ó 0 --> 0
**************************************************


Grabaré a un archivo crew.parquet este dataframe, por si en un futuro necesito alguna información adicional a los dataframes `actores_df` y `directores_df` que estoy creando

In [14]:
crew.write_parquet('data/crew.parquet')

En esta data `crew` aparecen registros por cada pelicula de todas las personas, que de una u otra forma han participado en ella (a excepción de actores que están en `cast`).  En la columna `job` se encuentra el rol que ha desempeñado cada una de estas personas.  La información que tenemos que filtrar es cuando en este campo aparece la palabra `DIRECTOR` y luego filtrar por director para sacar `información necesaria para satisfacer la función 6 de la Api` y `aportar data al modelo de recomendación`.  Los campos de la nueva tabla a crear serán:
- Nombre del Director: `director`
- Lista de los id de las peliculas en que participo: `movies`
- Número de películas en las que participó: `number_of_movies`
- El id del director: `id_director` --> Este campo por ahora creo no es necesario, me lo llevo por si lo necesito mas adelante, o mas adelante en la organizacion necesitan hacer un datawarehouse con esta información

In [15]:
# Esta Logica es similar a cuando obtuve la Data de `actores`
data_directores = []
mask = crew['job'] == 'DIRECTOR'
directores = crew.filter(mask)['name'].unique().to_list()

for director in directores:
    row_directores = {}
    mask = (crew['job'] == 'DIRECTOR') & (crew['name'] == director)
    id_director = crew.filter(mask)['id'].unique()[0]
    movies_ids = crew.filter(mask)['movie_id'].unique().to_list()
    number_of_movies = len(movies_ids)
    row_directores['id_director'] = id_director
    row_directores['director'] = director
    row_directores['movies'] = movies_ids
    row_directores['number_of_movies'] = number_of_movies
    data_directores.append(row_directores)

In [16]:
directores_df = pl.DataFrame(data=data_directores)

In [17]:
directores_df.head()

id_director,director,movies,number_of_movies
i64,str,list[i64],i64
234753,"""VIVIAN SCHILLING""",[60949],1
141696,"""HARI""","[49029, 62756]",2
1079373,"""BRAD BERNSTEIN""",[124069],1
1354959,"""JUSTIN TIPPING""",[385736],1
71609,"""JOACHIM TRIER""","[12197, 75233, 157827]",3


In [18]:
directores_df.write_parquet('data/directores.parquet')